Team Name: Pathfinder \
Team Member: Vignesh S S\
Problem Statememnt: AI-Based Computer Vision for Healthcare Hackathon\
Dataset Link: https://drive.google.com/drive/u/1/folders/1UVvAea975DxWIYFSULqmqVMKLquBOs1Y \
Landmark Prediction (Hypothesis-1)


In [1]:
# Install the library for segmentation models
!pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
import os

# Mount your Google Drive at /content/drive
drive.mount('/content/drive')

# --- Configuration ---
# IMPORTANT: Change this to the exact name of your zip file in Google Drive
ZIP_FILE_NAME = "/Hackathon_Dataset_20251025T071740Z.zip"
ZIP_PATH_ON_DRIVE = f"/content/{ZIP_FILE_NAME}"
UNZIP_DESTINATION = "/content/"  # Unzips to the fast local /content/ disk

# --- Unzip Logic ---
if os.path.exists(ZIP_PATH_ON_DRIVE):
    print(f"Found zip file at: {ZIP_PATH_ON_DRIVE}")
    print("Unzipping to local disk... This may take a minute.")
    # -q (quiet) suppresses the long list of all 200+ files
    !unzip -q "{ZIP_PATH_ON_DRIVE}" -d "{UNZIP_DESTINATION}"
    print("Unzipping complete!")

    # Verify the expected folder is now present
    print("Contents of /content/ (your unzipped data):")
    !ls "/content/"
else:
    print(f"ERROR: Zip file not found at: {ZIP_PATH_ON_DRIVE}")
    print("Please check the file name and its location in your Google Drive.")

Mounted at /content/drive
Found zip file at: /content//Hackathon_Dataset_20251025T071740Z.zip
Unzipping to local disk... This may take a minute.
Unzipping complete!
Contents of /content/ (your unzipped data):
 drive		      Hackathon_Dataset_20251025T071740Z.zip
'Hackathon Dataset'   sample_data


In [3]:
# --- 1. Imports ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Import the learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import cv2  # OpenCV for image processing
import numpy as np
import os
import glob  # For finding files
from tqdm.notebook import tqdm  # For progress bars
import albumentations as A
import segmentation_models_pytorch as smp


In [4]:
# --- 2. NEW: LandmarkDataset Class ---

def letterbox_image_landmark(image, target_size=(384, 384)):
    """
    Simpler letterbox: only processes the image.
    We'll process the landmarks (coordinates) separately.
    """
    h, w = image.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = int(w * scale), int(h * scale)

    resized_image = cv2.resize(
        image, (new_w, new_h), interpolation=cv2.INTER_LINEAR
    )

    top = (target_size[0] - new_h) // 2
    left = (target_size[1] - new_w) // 2
    bottom = target_size[0] - new_h - top
    right = target_size[1] - new_w - left

    padded_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right,
        borderType=cv2.BORDER_CONSTANT, value=0
    )

    # Return the padded image and the info needed to scale landmarks
    return padded_image, (scale, left, top)

class FetalLandmarkDataset(Dataset):
    """
    Custom Dataset for loading images and parsing .txt files
    for landmark coordinates.
    """
    def __init__(self, image_paths, label_paths, target_size=(384, 384),
                 augmentations=None):
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.target_size = target_size
        self.augmentations = augmentations
        self.class_map = {"cardiac": 0, "thorax": 1} # 2 classes

    def __len__(self):
        return len(self.image_paths)

    def get_landmarks_from_txt(self, label_path):
        """
        Parses the .txt file and returns the 4 key diameter points:
        [cardiac_L_x, cardiac_R_x, thorax_L_x, thorax_R_x]
        We will simplify and just predict the horizontal extents.
        """
        # Store ellipse params: [center_x, major_axis]
        ellipse_data = {}

        try:
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 6: continue

                    class_name = parts[5]
                    class_idx = self.class_map.get(class_name)
                    if class_idx is not None:
                        # We only care about x-center and x-radius
                        center_x = float(parts[0])
                        major_axis = float(parts[2]) # Full axis
                        ellipse_data[class_name] = (center_x, major_axis)

            # Now, build the 4-point landmark tensor
            # [cardiac_L_x, cardiac_R_x, thorax_L_x, thorax_R_x]
            # This is a simplification. A more complex way would involve rotation,
            # but for a time-limited hackathon, this is a smart, fast hypothesis.

            cardiac_c, cardiac_a = ellipse_data.get("cardiac", (0.0, 0.0))
            thorax_c, thorax_a = ellipse_data.get("thorax", (0.0, 0.0))

            # Calculate left and right extents
            cardiac_L_x = cardiac_c - (cardiac_a / 2)
            cardiac_R_x = cardiac_c + (cardiac_a / 2)
            thorax_L_x = thorax_c - (thorax_a / 2)
            thorax_R_x = thorax_c + (thorax_a / 2)

            # For simplicity, we'll just predict these 4 x-values
            landmarks = [cardiac_L_x, cardiac_R_x, thorax_L_x, thorax_R_x]
            return np.array(landmarks, dtype=np.float32)

        except Exception as e:
            # On error (e.g., missing file), return zeros
            print(f"Error processing landmark file {label_path}: {e}")
            return np.zeros(4, dtype=np.float32)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = self.label_paths[idx]

        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            image = np.zeros(self.target_size, dtype=np.uint8) # Failsafe

        # Get original landmarks
        landmarks = self.get_landmarks_from_txt(label_path)

        # Get original image shape
        orig_h, orig_w = image.shape[:2]

        # Apply letterbox padding to image
        image, (scale, pad_left, pad_top) = letterbox_image_landmark(
            image, self.target_size
        )

        # --- Scale Landmarks ---
        # 1. Scale with image
        landmarks *= scale
        # 2. Add padding offset
        landmarks += pad_left
        # 3. Normalize landmarks to [0, 1] range relative to image size
        landmarks_normalized = landmarks / self.target_size[1] # Divide by width (384)

        # Apply augmentations (image only)
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']
            # Note: We don't augment the landmarks here for simplicity.
            # A more robust pipeline would use A.Keypoints

        # Convert to Tensors
        image = torch.from_numpy(image).float().unsqueeze(0)
        landmarks_tensor = torch.from_numpy(landmarks_normalized).float()

        image = image / 255.0

        return image, landmarks_tensor

# --- 3. NEW: Landmark Model ---
class LandmarkUNet(nn.Module):
    """
    This model uses the pre-trained U-Net encoder but replaces the
    decoder with a simple regression head to predict 4 numbers.
    """
    def __init__(self, n_outputs=4):
        super().__init__()

        # Load the pre-trained segmentation model
        # We will steal its encoder!
        self.base_model = smp.Unet(
            encoder_name="resnet34",
            encoder_weights="imagenet",
            in_channels=1,
            classes=3, # Doesn't matter, we're removing this part
        )

        # Get the encoder
        self.encoder = self.base_model.encoder

        # Define our new "head"
        # The resnet34 encoder's final output has 512 channels
        # After global pooling, this becomes a vector of size 512
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.head = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, n_outputs)
            # We don't use Sigmoid, as values are just normalized [0, 1]
            # MSELoss works better on raw logits.
        )

    def forward(self, x):
        # Pass image through the encoder
        features = self.encoder(x)

        # We only care about the final, deepest feature map
        x = features[-1]

        # Run it through our new head
        x = self.pooling(x)
        x = self.flatten(x)
        x = self.head(x)
        return x

In [5]:
#Landmark Training Script

# --- 1. Set Up Paths ---
# (This is the same as before, from our corrected script)
BASE_DATA_PATH = "/content/Hackathon Dataset/"
TRAIN_DIR = os.path.join(BASE_DATA_PATH, "Images/training")
VAL_DIR = os.path.join(BASE_DATA_PATH, "Images/validation")

train_img_paths = sorted(glob.glob(os.path.join(TRAIN_DIR, "images", "*.png")))
train_lbl_paths = [
    p.replace("/images/", "/annfiles_ellipse/").replace(".png", ".txt")
    for p in train_img_paths
]
val_img_paths = sorted(glob.glob(os.path.join(VAL_DIR, "images", "*.png")))
val_lbl_paths = [
    p.replace("/images/", "/annfiles_ellipse/").replace(".png", ".txt")
    for p in val_img_paths
]

# --- 2. Augmentations (Image-only) ---
# We don't need the complex geometric ones since we aren't
# augmenting the keypoints (for speed).
train_augs_land = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.3),
    A.MotionBlur(p=0.3),
])
val_augs_land = None

# --- 3. Create Datasets & DataLoaders ---
TARGET_IMG_SIZE = (384, 384)

train_dataset_land = FetalLandmarkDataset(
    train_img_paths, train_lbl_paths,
    target_size=TARGET_IMG_SIZE,
    augmentations=train_augs_land
)
val_dataset_land = FetalLandmarkDataset(
    val_img_paths, val_lbl_paths,
    target_size=TARGET_IMG_SIZE,
    augmentations=val_augs_land
)

BATCH_SIZE = 8
train_loader_land = DataLoader(
    train_dataset_land, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=2, pin_memory=True
)
val_loader_land = DataLoader(
    val_dataset_land, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, pin_memory=True
)

# --- 4. Model, Loss, Optimizer ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_EPOCHS = 25
LEARNING_RATE = 1e-4

# Instantiate our new model
model_land = LandmarkUNet(n_outputs=4).to(DEVICE)

# --- NEW: Use MSELoss ---
loss_fn_land = nn.MSELoss() # Mean Squared Error Loss

optimizer_land = torch.optim.Adam(model_land.parameters(), lr=LEARNING_RATE)

scheduler_land = ReduceLROnPlateau(
    optimizer_land, 'min', patience=3, factor=0.1
)

# --- 5. Model Save Path ---
DRIVE_PATH = "/content/drive/My Drive/"
SAVE_PATH_LAND = os.path.join(
    DRIVE_PATH,
    "hackathon_submission/task_1_landmark/Model Weights/best_model_hypothesis_1.pth"
)
os.makedirs(os.path.dirname(SAVE_PATH_LAND), exist_ok=True)

best_val_loss = float('inf') # Track lowest loss

# --- 6. Training Loop ---
print(f"--- Starting Landmark Training on {DEVICE} ---")

for epoch in range(NUM_EPOCHS):
    model_land.train()
    train_loss = 0.0

    loop = tqdm(train_loader_land, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
    for images, landmarks in loop:
        images, landmarks = images.to(DEVICE), landmarks.to(DEVICE)

        optimizer_land.zero_grad()
        outputs = model_land(images)
        loss = loss_fn_land(outputs, landmarks)
        loss.backward()
        optimizer_land.step()

        train_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    # --- Validation ---
    model_land.eval()
    val_loss = 0.0

    loop_val = tqdm(val_loader_land, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]")
    with torch.no_grad():
        for images, landmarks in loop_val:
            images, landmarks = images.to(DEVICE), landmarks.to(DEVICE)
            outputs = model_land(images)
            loss = loss_fn_land(outputs, landmarks)
            val_loss += loss.item()
            loop_val.set_postfix(val_loss=loss.item())

    avg_train_loss = train_loss / len(train_loader_land)
    avg_val_loss = val_loss / len(val_loader_land)

    # We don't have IoU here, just loss
    print(f"Epoch {epoch+1} -> Train Loss: {avg_train_loss:.6f}, "
          f"Val Loss: {avg_val_loss:.6f}")

    scheduler_land.step(avg_val_loss)

    # --- Save based on best *loss* ---
    if avg_val_loss < best_val_loss:
        print(f"Validation loss improved ({best_val_loss:.6f} -> {avg_val_loss:.6f}). "
              f"Saving model...")
        best_val_loss = avg_val_loss
        torch.save(model_land.state_dict(), SAVE_PATH_LAND)

print("--- Landmark Training Finished ---")
print(f"Best model (Val Loss: {best_val_loss:.6f}) saved to {SAVE_PATH_LAND}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

--- Starting Landmark Training on cuda ---


Epoch 1/25 [Train]:   0%|          | 0/25 [00:00<?, ?it/s]

...

Error processing landmark file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt'
Error processing landmark file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt'
Epoch 24 -> Train Loss: 0.005171, Val Loss: 0.019761


Epoch 25/25 [Train]:   0%|          | 0/25 [00:00<?, ?it/s]

Error processing landmark file /content/Hackathon Dataset/Images/training/annfiles_ellipse/189.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/189.txt'
Error processing landmark file /content/Hackathon Dataset/Images/training/annfiles_ellipse/047.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/047.txt'
Error processing landmark file /content/Hackathon Dataset/Images/training/annfiles_ellipse/108.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/108.txt'


Epoch 25/25 [Val]:   0%|          | 0/7 [00:00<?, ?it/s]

Error processing landmark file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt'
Error processing landmark file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt'
Epoch 25 -> Train Loss: 0.004328, Val Loss: 0.019755
--- Landmark Training Finished ---
Best model (Val Loss: 0.019316) saved to /content/drive/My Drive/hackathon_submission/task_1_landmark/Model Weights/best_model_hypothesis_1.pth
